<a href="https://colab.research.google.com/github/chcorophyll/my_deeplearning_cookbook/blob/master/my_Generating_Text_in_the_Style_of_an_Example_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/chcorophyll/deep_learning_cookbook.git

Cloning into 'deep_learning_cookbook'...
remote: Enumerating objects: 427, done.
remote: Total 427 (delta 0), reused 0 (delta 0), pack-reused 427
Receiving objects: 100% (427/427), 160.26 MiB | 34.73 MiB/s, done.
Resolving deltas: 100% (207/207), done.
Checking out files: 100% (86/86), done.


In [0]:
!ls

deep_learning_cookbook	sample_data


In [0]:
import os

path_org = os.getcwd()
data_path = os.path.join(path_org, "deep_learning_cookbook")
os.chdir(data_path)

In [0]:
data_path

'/content/deep_learning_cookbook'

In [0]:
!ls

'03.1 Using pre trained word embeddings.ipynb'
'03.2 Domain specific ranking using word2vec cosine distance.ipynb'
'04.1 Collect movie data from Wikipedia.ipynb'
'04.2 Build a recommender system based on outgoing Wikipedia links.ipynb'
'05.1 Generating Text in the Style of an Example Text.ipynb'
'06.1 Question matching.ipynb'
'07.1 Text Classification.ipynb'
'07.2 Emoji Suggestions.ipynb'
'07.3 Tweet Embeddings.ipynb'
'08.1 Sequence to sequence mapping.ipynb'
'08.2 Import Gutenberg.ipynb'
'08.3 Subword tokenizing.ipynb'
'09.1 Reusing a pretrained image recognition network.ipynb'
'09.2 Images as embeddings.ipynb'
'09.3 Retraining.ipynb'
'10.1 Building an inverse image search service.ipynb'
'11.1 Detecting Multiple Images.ipynb'
'12.1 Activation Optimization.ipynb'
'12.2 Neural Style.ipynb'
'13.1 Quick Draw Cat Autoencoder.ipynb'
'13.2 Variational Autoencoder.ipynb'
'13.5 Quick Draw Autoencoder.ipynb'
'14.1 Importing icons.ipynb'
'14.2 Icon Autoencoding.ipynb'
'14.2 Variational Autoencod

In [3]:
!sudo apt-get install libdb++-dev
!export BERKELEYDB_DIR=/usr
!pip install gutenberg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libdb5.3 libdb5.3++ libdb5.3++-dev libdb5.3-dev
Suggested packages:
  db5.3-doc
The following NEW packages will be installed:
  libdb++-dev libdb5.3++ libdb5.3++-dev libdb5.3-dev
The following packages will be upgraded:
  libdb5.3
1 upgraded, 4 newly installed, 0 to remove and 15 not upgraded.
Need to get 2,918 kB of archives.
After this operation, 8,395 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libdb5.3 amd64 5.3.28-13.1ubuntu1.1 [672 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libdb5.3++ amd64 5.3.28-13.1ubuntu1.1 [703 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libdb5.3-dev amd64 5.3.

In [4]:
!pip install gutenberg

In [0]:
try:
    GUTENBERG = True
    from gutenberg.acquire import load_etext
    from gutenberg.query import get_etexts, get_metadata
    from gutenberg.acquire import get_metadata_cache
    from gutenberg.acquire.text import UnknownDownloadUriException
    from gutenberg.cleanup import strip_headers
    from gutenberg._domain_model.exceptions import CacheAlreadyExistsException
except ImportError:
    GUTENBERG = False
    print('Gutenberg is not installed. See instructions at https://pypi.python.org/pypi/Gutenberg')
from keras.models import Input, Model
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers.wrappers import TimeDistributed
import keras.callbacks
import keras.backend as K
import scipy.misc
import json

import os, sys
import re
import PIL
from PIL import ImageDraw

from keras.optimizers import RMSprop
import random
import numpy as np
import tensorflow as tf
from keras.utils import get_file

from IPython.display import clear_output, Image, display, HTML
try:
    from io import BytesIO
except ImportError:
    from StringIO import StringIO as BytesIO

In [0]:
if GUTENBERG:
    cache = get_metadata_cache()
    try:
        cache.populate()
    except CacheAlreadyExistsException:
        pass

KeyboardInterrupt: ignored

In [0]:
if GUTENBERG:
    for text_id in get_etexts("author", "Shakespeare, William"):
        print(text_id, list(get_metadata("title", text_id))[0])

In [6]:
if GUTENBERG:
    shakespeare = strip_headers(load_etext(100))
else:
    path = get_file('shakespeare', 'https://storage.googleapis.com/deep-learning-cookbook/100-0.txt')
    shakespeare = open(path).read()
training_text = shakespeare.split('\nTHE END', 1)[-1]
len(training_text)

5466234

In [7]:
chars = list(sorted(set(training_text)))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
len(chars)

101

In [0]:
def char_rnn_model(num_chars, num_layers, num_nodes=512, dropout=0.1):
    input = Input(shape=(None, num_chars), name="input")
    prev = input
    for i in range(num_layers):
        lstm = LSTM(num_nodes, return_sequences=True, name="lstm_layer_%d" %(i + 1))(prev)
        if dropout:
            prev = Dropout(dropout)(lstm)
        else:
            prev = lstm
    dense = TimeDistributed(Dense(num_chars, name="dense", activation="softmax"))(prev)
    model = Model(inputs=[input], outputs=[dense])
    optimizer = RMSprop(lr=0.01)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [9]:
model = char_rnn_model(len(chars), num_layers=2, num_nodes=640, dropout=0)
model.summary()

W0625 00:29:31.145601 139834438612864 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0625 00:29:31.187596 139834438612864 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0625 00:29:31.189401 139834438612864 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0625 00:29:34.001946 139834438612864 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0625 00:29:34.012756 139834438612864 deprecation_wrappe

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, None, 101)         0         
_________________________________________________________________
lstm_layer_1 (LSTM)          (None, None, 640)         1899520   
_________________________________________________________________
lstm_layer_2 (LSTM)          (None, None, 640)         3279360   
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 101)         64741     
Total params: 5,243,621
Trainable params: 5,243,621
Non-trainable params: 0
_________________________________________________________________


In [0]:
(101 * 640 + 640*640)*4 + 640*4 

1899520

In [10]:
CHUNK_SIZE = 160

def data_generator(all_text, char_to_idx, batch_size, chunk_size):
    X = np.zeros((batch_size, chunk_size, len(char_to_idx)))
    y = np.zeros((batch_size, chunk_size, len(char_to_idx)))
    while True:
        for row in range(batch_size):
            idx = random.randrange(len(all_text) - chunk_size - 1)
            chunk = np.zeros((chunk_size + 1, len(char_to_idx)))
            for i in range(chunk_size + 1):
                chunk[i, char_to_idx[all_text[idx + i]]] = 1
            X[row, :, :] = chunk[:chunk_size]
            y[row, :, :] = chunk[1:]
        yield X, y

next(data_generator(training_text, char_to_idx, 4, chunk_size=CHUNK_SIZE))

(array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 1., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0

In [11]:
early = keras.callbacks.EarlyStopping(monitor='loss',
                              min_delta=0.03,
                              patience=3,
                              verbose=0, mode='auto')

BATCH_SIZE = 256
model.fit_generator(
    data_generator(training_text, char_to_idx, batch_size=BATCH_SIZE, chunk_size=CHUNK_SIZE),
    epochs=40,
    callbacks=[early,],
    steps_per_epoch=2 * len(training_text) / (BATCH_SIZE * CHUNK_SIZE),
    verbose=2
)

W0625 00:35:09.789474 139834438612864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0625 00:35:11.030517 139834438612864 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/40
 - 179s - loss: 3.5777 - acc: 0.1605
Epoch 2/40
 - 176s - loss: 3.3022 - acc: 0.1952
Epoch 3/40
 - 177s - loss: 3.2754 - acc: 0.1971
Epoch 4/40
 - 182s - loss: 2.9162 - acc: 0.2628
Epoch 5/40
 - 185s - loss: 2.4996 - acc: 0.3863
Epoch 6/40
 - 185s - loss: 2.4097 - acc: 0.4155
Epoch 7/40
 - 185s - loss: 2.3806 - acc: 0.4245
Epoch 8/40
 - 185s - loss: 2.3580 - acc: 0.4310
Epoch 9/40
 - 184s - loss: 2.3470 - acc: 0.4351
Epoch 10/40
 - 185s - loss: 2.3580 - acc: 0.4337
Epoch 11/40
 - 184s - loss: 2.3123 - acc: 0.4449
Epoch 12/40
 - 184s - loss: 2.3162 - acc: 0.4456
Epoch 13/40
 - 184s - loss: 2.3426 - acc: 0.4393
Epoch 14/40
 - 184s - loss: 2.2972 - acc: 0.4504


In [0]:
with open('zoo/06/shakespeare.json', 'w') as fout:
    json.dump({
        'chars': ''.join(chars),
        'char_to_idx': char_to_idx,
        'chunk_size': CHUNK_SIZE,
    }, fout)
model.save('zoo/06/shakespeare.h5')

In [17]:
def generate_output(model, training_text, start_index=None, diversity=None, amount=400):
    if start_index is None:
        start_index = random.randint(0, len(training_text) - CHUNK_SIZE - 1)
    generated = training_text[start_index: start_index + CHUNK_SIZE]
    yield generated + '#'
    for i in range(amount):
        x = np.zeros((1, len(generated), len(chars)))
        for t, char in enumerate(generated):
            x[0, t, char_to_idx[char]] = 1.
        preds = model.predict(x, verbose=0)[0]
        
        if diversity is None:
            next_index = np.argmax(preds[len(generated) - 1])
        else:
            preds = np.asarray(preds[len(generated) - 1]).astype('float64')
            preds = np.log(preds) / diversity
            exp_preds = np.exp(preds)
            preds = exp_preds / np.sum(exp_preds)
            probas = np.random.multinomial(1, preds, 1)
            next_index = np.argmax(probas)     
        next_char = chars[next_index]
        yield next_char

        generated += next_char
    return generated

for ch in generate_output(model, training_text):
    sys.stdout.write(ch)
print()

 gilt copper
       crown.
    6. MARQUIS DORSET, bearing a sceptre of gold, on his head a
       demi-coronal of gold. With him, the EARL OF SURREY,
       bea#ring the King of                                                                                                                                                                                                                                                                                                                                                                                                


In [21]:
def generate_output(model, training_text, start_index=None, diversity=None, amount=400):
    if start_index is None:
        start_index = random.randint(0, len(training_text) - CHUNK_SIZE - 1)
    fragment = training_text[start_index: start_index + CHUNK_SIZE]
    generated = fragment
    for i in range(amount):
        x = np.zeros((1, CHUNK_SIZE, len(chars)))
        for t, char in enumerate(fragment):
            x[0, t, char_to_idx[char]] = 1.
        preds = model.predict(x, verbose=0)[0]
        preds = np.asarray(preds[len(fragment)-1])
        next_index = np.argmax(preds)
        next_char = chars[next_index]
        generated += next_char
        fragment = fragment[1:] + next_char
    return generated
        

for line in generate_output(model, training_text).split("\n"):
    print(line)

 traitor?
    Sword, I will hallow thee for this thy deed
    And hang thee o'er my tomb when I am dead.
    Ne'er shall this blood be wiped from thy point,
    The sea is born and strong and seen the state of this to thee.
                                                                                                                                                                                                                                                                                                                                                


In [24]:
def find_python(rootdir):
    matches = []
    for root, dirnames, filenames in os.walk(rootdir):
        for fn in filenames:
            if fn.endswith('.py'):
                matches.append(os.path.join(root, fn))

    return matches
#  + find_python(os.path.join(sys.executable.rsplit('/', 2)[0], 'lib'))
srcs = find_python(random.__file__.rsplit('/', 1)[0])
len(srcs)

625

In [0]:
# delete review
# COMMENT_RE = re.compile("#.*")
# srcs = COMMENT_RE.sub(" ", srcs)

In [27]:
def replacer(value):
    value = "".join(ch for ch in value if ord(ch) < 127)
    if not " " in value:
        return value
    if sum(1 for ch in value if ch.isalpha()) > 6:
        return "MSG"
    return value

def replace_literals(st):
    res = []
    start_text = start_quote = i = 0
    quote = ''
    while i < len(st):
        if quote:
            if st[i: i + len(quote)] == quote:
                quote = ''
                start_text = i
                res.append(replacer(st[start_quote: i]))
        elif st[i] in '"\'':
            quote = st[i]
            if i < len(st) - 2 and st[i + 1] == st[i + 2] == quote:
                quote = 3 * quote
            start_quote = i + len(quote)
            res.append(st[start_text: start_quote])
        if st[i] == '\n' and len(quote) == 1:
            start_text = i
            res.append(quote)
            quote = ''
        if st[i] == '\\':
            i += 1
        i += 1
    return ''.join(res) + st[start_text:]

#replace_literals('print("hel\\"lo")') + replace_literals("print('hel\\'lo world')")
replace_literals('this = "wrong\n')

'this = ""\n'

In [28]:
COMMENT_RE = re.compile('#.*')
python_code = []
for fn in srcs:
    try:
        with open(fn, 'r') as fin:
            src = fin.read()
    except UnicodeDecodeError:
        print('Could not read %s' % fn)
    src = replace_literals(src)
    src = COMMENT_RE.sub('', src)
    python_code.append(src)

python_code = '\n\n\n'.join(python_code)
len(python_code)

6332828

In [29]:
py_chars = list(sorted(set(python_code)))
py_char_to_idx = {ch: idx for idx, ch in enumerate(py_chars)}
len(py_chars)

97

In [30]:
py_model = char_rnn_model(len(py_chars), num_layers=2, num_nodes=640, dropout=0)
py_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, None, 97)          0         
_________________________________________________________________
lstm_layer_1 (LSTM)          (None, None, 640)         1889280   
_________________________________________________________________
lstm_layer_2 (LSTM)          (None, None, 640)         3279360   
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 97)          62177     
Total params: 5,230,817
Trainable params: 5,230,817
Non-trainable params: 0
_________________________________________________________________


In [31]:
early = keras.callbacks.EarlyStopping(monitor='loss',
                              min_delta=0.03,
                              patience=3,
                              verbose=0, mode='auto')

BATCH_SIZE = 256
py_model.fit_generator(
    data_generator(python_code, py_char_to_idx, batch_size=BATCH_SIZE, chunk_size=160),
    epochs=40,
    callbacks=[early,],
    steps_per_epoch=2 * len(python_code) / (BATCH_SIZE * 160),
    verbose=2
)

Epoch 1/40
 - 203s - loss: 3.1561 - acc: 0.3511
Epoch 2/40
 - 209s - loss: 2.9980 - acc: 0.3578
Epoch 3/40
 - 207s - loss: 3.0062 - acc: 0.3552
Epoch 4/40
 - 207s - loss: 3.0315 - acc: 0.3555
Epoch 5/40
 - 207s - loss: 3.0135 - acc: 0.3572


In [34]:
def generate_code(model, start_with='\ndef ', end_with='\n\n', diversity=1.0):
    generated = start_with
    yield generated
    for i in range(2000):
        x = np.zeros((1, len(generated), len(py_chars)))
        for t, char in enumerate(generated):
            x[0, t, py_char_to_idx[char]] = 1.
        preds = model.predict(x, verbose=0)[0]
        
        preds = np.asarray(preds[len(generated) - 1]).astype('float64')
        preds = np.log(preds) / diversity
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        next_index = np.argmax(probas)        
        next_char = py_chars[next_index]
        yield next_char

        generated += next_char
        if generated.endswith(end_with):
            break

for i in range(20):
    for ch in generate_code(py_model):
        sys.stdout.write(ch)
#         st += ch
    print()


def 
Ue"x S.e Hre _S )t  e Nlioesae=i na, r o ye i. tf"mp  tnnpt 
 _  ' d" (u l ait sN"oti
r +eeiwe iek tis   ofn(srs
oc ar_e,tn
 _ t  _ "tlb_ rf etie ioe
 r)'= dnes:-it 
   fue=ld_ omebm+c(
 r"err Fl + x0e t nt=n 2 
 _b ca_S ctUeeol. fs   drf< nr   l snsr ,c_ni0e_Tm a
ot  sed
ii tGcp=d)' dec  =e,  etr _-,  :3G s_e : n" ess
  aetfoe sua
 , fe:
 'tIu 8n_ =tiet) rth (7 t= or=ac_   wf'D2c
rb rrmn i\_
c _ o gfpo i,)hg r')has 
tc d.t_ 4c
   :' f = e
 Rt t") o'r (e sD m
r 
da_n  r
R i  tczte     f sa   : oapt n  lTd' t  l_s4i  s=pa_s ns
   
o c ,u =sts  en  u 



def .__n i l  yd   are x V  _ ou si  i
:l)e or a  slp  n_odge cn ' ntv_aiia en Ix._e = l S_c ey
7 ad  am
m  eii  efhd  (  y    E
    ns  eT mi ofE=_  l  M (tc))  ps a  
rB(  os_co oss  



def t''e lfmndtg,n elog '
)e  mr s    
t_at D"a Kc\  f nri   afd  :s 
pm"C m yasis  
e  rnebG .S sr'"mr:      ib  t  '    .Xi9eo=  it 
  g    cten l
"s_  0lf :te =('xl6x p  f  r o_
 _  Irg=n 
 l) :=   
e, e     = a a <af [ 0ssK t ) 
Me",eni
  rne

KeyboardInterrupt: ignored

In [35]:

BATCH_SIZE = 512

flat_model = char_rnn_model(len(py_chars), num_layers=1, num_nodes=512, dropout=0)

early = keras.callbacks.EarlyStopping(monitor='loss',
                              min_delta=0.03,
                              patience=3,
                              verbose=0, mode='auto')

flat_model.fit_generator(
    data_generator(python_code, py_char_to_idx, batch_size=BATCH_SIZE, chunk_size=160),
    epochs=40,
    callbacks=[early,],
    steps_per_epoch=2 * len(python_code) / (BATCH_SIZE * 160),
    verbose=2
)

Epoch 1/40
 - 62s - loss: 3.8293 - acc: 0.2209
Epoch 2/40
 - 61s - loss: 3.6845 - acc: 0.2358
Epoch 3/40
 - 62s - loss: 3.6464 - acc: 0.2414
Epoch 4/40
 - 62s - loss: 3.5930 - acc: 0.2530
Epoch 5/40
 - 62s - loss: 3.3954 - acc: 0.2888
Epoch 6/40
 - 63s - loss: 2.8279 - acc: 0.3750
Epoch 7/40
 - 64s - loss: 2.3901 - acc: 0.4486
Epoch 8/40
 - 64s - loss: 2.2107 - acc: 0.4998
Epoch 9/40
 - 64s - loss: 2.2179 - acc: 0.4995
Epoch 10/40
 - 64s - loss: 2.1801 - acc: 0.5102
Epoch 11/40
 - 64s - loss: 2.0979 - acc: 0.5331
Epoch 12/40
 - 64s - loss: 2.1189 - acc: 0.5282
Epoch 13/40
 - 64s - loss: 2.0703 - acc: 0.5416
Epoch 14/40
 - 64s - loss: 2.0791 - acc: 0.5400


In [36]:
example_code = 'if a == 2:\n    b=1\nelse:\n    b=2\n'
#example_code = 'a=(2 * 3)\nb=(4 * 6 + 7)\nreturn C'

def activations(model, code):
    x = np.zeros((1, len(code), len(py_char_to_idx)))
    for t, char in enumerate(code):
        x[0, t, py_char_to_idx[char]] = 1.
    output = model.get_layer('lstm_layer_1').output
    f = K.function([model.input], [output])
    return f([x])[0][0]

act = activations(flat_model, example_code)
act.shape

(33, 512)

In [37]:
len(example_code)

33

In [38]:
def interesting_neurons(act):
    res = []
    for n in np.argmax(act, axis=1):
        if not n in res:
            res.append(n)
    return res

neurons = interesting_neurons(act)
len(neurons)

3

In [47]:
!pip3 install Pillow

In [53]:
def visualize_neurons(neurons, code, act, cell_size=12):
    img = np.full((len(neurons) + 1, len(code), 3), 128)
    scores = (act[:, neurons].T + 1) / 2
    img[1:, :, 0] = 255 * (1 - scores)
    img[1:, :, 1] = 255 * scores

    f = BytesIO()
    img = scipy.misc.imresize(img, float(cell_size), interp='nearest')
#     pil_img = np.array(PIL.Image.fromarray(img).resize(cell_size, ))
    pil_img = PIL.Image.fromarray(img)
    draw = ImageDraw.Draw(pil_img)
    for idx, ch in enumerate(code):
        draw.text((idx * cell_size + 2, 0), ch)
    pil_img.save(f, 'png')
    return Image(data=f.getvalue())

img = visualize_neurons(neurons, example_code, act)
display(img)

AttributeError: ignored

In [0]:
def image_for_code(code):
    act = activations(flat_model, code)
    neurons = interesting_neurons(act)
    return visualize_neurons(neurons, code, act)

display(image_for_code('if (a == 2) and ((b == 1) or (c==2)):'))

In [0]:
code = 'if (a == 2) and ((b == 1) or (c==2)):'
mask = '   ________     ____________________ '
act = activations(flat_model, code)
positive = [idx for idx, ch in enumerate(mask) if ch == '_']
negative = [idx for idx, ch in enumerate(mask) if ch != '_']

neurons = np.argsort(act[positive].sum(axis=0) - act[negative].sum(axis=0))[-5:]

In [0]:
img = visualize_neurons(neurons, code, act)
display(img)

In [0]:
neurons

In [0]:
act[negative, 108].sum()

In [0]:
x0 = 0
x1 = 0
for idx, ch in enumerate(mask):
    if ch == '_':
        x0 += act[idx, 108]
    else:
        x1 += act[idx, 108]
x0, x1